In [1]:
import pandas as pd
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.models import BasicTickFormatter, Label, Title, NumeralTickFormatter
# HoverTool allows you to set a tooltips property which takes a list of tuples
from bokeh.models.tools import HoverTool
# a pre-made five color pallette, one of Bokeh’s many pre-made color palettes
from bokeh.palettes import TolRainbow, Category20c, Spectral
# a helper method for mapping colors to bars in a bar-charts
from bokeh.transform import factor_cmap
import math

datapath = 'datapandas/'
inputFile = 'ABS2021EduInstBySex.xlsx'     # population count by State by AgeGroup by Sex

# Function "output_file" defines the output name and how the visualization will be rendered by bokeh, eg html for this one.
# File will be saved in the main project folder
output_file('codeflask/templates/educationByInstBySex.html')

# Read input excel file
dfEduInstBySex = pd.read_excel(datapath+inputFile)
#print(dfEduInstBySex,'\n')
#   Institution   Males  Females State

# Group data by Institution
dfGrouped = dfEduInstBySex.groupby('Institution')[['Males','Females']].sum().reset_index()
#print(dfGrouped,'\n')

# Re-order data for Graph purposes
df1 = dfGrouped.iloc[1:4]
df2 = dfGrouped.iloc[[5]]
df3 = dfGrouped.iloc[[4]]
df4 = dfGrouped.iloc[[0]]
dfGrouped=pd.concat([df1,df2,df3,df4], ignore_index = True)  # Concatenate dataframes
#print(dfGrouped,'\n')
#  Institution    Males  Females
#0   Preschool   252528   231655
#1     Primary  1064523  1010691
#2   Secondary   827184   802449
#3  Vocational   275119   326784
#4  University   495015   690430
#5       Other   106197   136627

# Set dataframe into 'column data source' so it can be processed by bokeh
source = ColumnDataSource(dfGrouped)

# Get list of Categorical 'Institution' values for Stacked bar chart (x-axis)
InstDesc = source.data['Institution'].tolist()
#print(InstDesc,'\n')
Gender = ['Males','Females']

cols = ['#add836', '#ffa074']   #, '#00cc99'

#color_map = factor_cmap(field_name='Institution',
#                        palette=Spectral6,
#                        factors=InstDesc)
#

p = figure(x_range=InstDesc, width=700, height=400
           , title="Type of Education attending by sex as at 2021 Census"
           , background_fill_color="#292929" #404040
           , border_fill_color = "black"
           , outline_line_color= "white"
           , outline_line_width = 0
           )

p.vbar_stack(Gender,
               x='Institution',
               source=source,
               color=cols,
               width=0.5,
               legend_label=Gender
               #fill_color=color_map
             )

# Add labels
p.xaxis.axis_label = 'Educational Institution'
p.yaxis.axis_label = 'Population'

#------------------------------------------------------------------------

# Change scientific notation in y-axis
p.yaxis[0].formatter = NumeralTickFormatter(format="0,0")

#--------------------------------------------------------------------------
# Add hover
hover = HoverTool()    # HoverTool allows you to set a tooltips property which takes a list of tuples
hover.tooltips = [("","Males:@Males{0,0} Females:@Females{0,0}")]
#hover.mode = 'vline'   # tell the popup to show when a vertical line crosses a glyph
p.add_tools(hover)     # add hover tool in the toolbar.

#show(p)
save(p)

#end of file